In [1]:
# ✅ Install required libraries
!pip install requests pandas folium

import requests
import pandas as pd
import folium

# ✅ Function to Fetch Traffic Data (OpenStreetMap Overpass API)
def get_traffic_data():
    print("🔄 Fetching traffic data...")
    overpass_url = "http://overpass-api.de/api/interpreter"
    query = "[out:json];node(around:5000,12.9716,77.5946)[highway=traffic_signals];out;"

    response = requests.get(overpass_url, params={'data': query})

    if response.status_code == 200:
        data = response.json()
        traffic_data = [{'lat': node['lat'], 'lon': node['lon']} for node in data.get('elements', [])]
        return traffic_data
    else:
        print("❌ Error fetching traffic data:", response.status_code, response.text)
        return []

# ✅ Function to Fetch Weather Data (MET Norway API)
def get_weather_data():
    print("🔄 Fetching weather data...")
    weather_url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
    headers = {"User-Agent": "ColabDataIntegration/1.0"}
    params = {'lat': 12.9716, 'lon': 77.5946}  # Bangalore Coordinates

    response = requests.get(weather_url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        weather_data = {
            'temperature': data['properties']['timeseries'][0]['data']['instant']['details']['air_temperature'],
            'humidity': data['properties']['timeseries'][0]['data']['instant']['details']['relative_humidity'],
            'wind_speed': data['properties']['timeseries'][0]['data']['instant']['details']['wind_speed']
        }
        return weather_data
    else:
        print("❌ Error fetching weather data:", response.status_code, response.json())
        return {}

# ✅ Fetch & Store Data
traffic_data = get_traffic_data()
weather_data = get_weather_data()

# ✅ Convert to Pandas DataFrame
traffic_df = pd.DataFrame(traffic_data)
weather_df = pd.DataFrame([weather_data])  # Convert single dict to DataFrame

# ✅ Display Data
print("🚦 Traffic Data:")
print(traffic_df.head())

print("\n🌤 Weather Data:")
print(weather_df)

# ✅ Visualize Traffic Data on a Map
if not traffic_df.empty:
    map_bangalore = folium.Map(location=[12.9716, 77.5946], zoom_start=12)

    for _, row in traffic_df.iterrows():
        folium.Marker(location=[row['lat'], row['lon']], icon=folium.Icon(color='red')).add_to(map_bangalore)

    map_bangalore.save("traffic_map.html")
    print("🗺 Map saved as 'traffic_map.html'. Open it in a browser!")


🔄 Fetching traffic data...
🔄 Fetching weather data...
🚦 Traffic Data:
         lat        lon
0  12.974611  77.620045
1  12.981592  77.623220
2  12.966119  77.606762
3  12.958025  77.605982
4  12.959934  77.601917

🌤 Weather Data:
   temperature  humidity  wind_speed
0         33.5      13.9         5.4
🗺 Map saved as 'traffic_map.html'. Open it in a browser!
